In [1]:
from IPython.core.display import HTML
import requests
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import datetime
import seaborn as sns
from dateutil import parser
import pytz
import pylab as pl
from pyquery import PyQuery as pq
import numpy as np
import scipy
import pandas as pd
from lxml import etree
import os
from os import path
from pprint import pprint
import dateutil
import json
%matplotlib inline

In [2]:
df = pd.read_csv(path.join('scraped-data', 'records-all-with-info.csv'), 
                 encoding='utf-8')
df_member = pd.read_csv(path.join('research-data', 'member-profile-2012-2016-en.csv'), 
                 encoding='utf-8')
df_party_to_camp_mapping = pd.read_csv(path.join('research-data', 'party-to-camp-mapping-en.csv'), 
                 encoding='utf-8')

from datetime import date

df['date'] = df['vote-date'].apply(lambda x: date(
        int(x.split('/')[2]), 
        int(x.split('/')[1]), 
        int(x.split('/')[0])
    ))
df['datetime'] = df['date'].apply(lambda d: datetime.datetime(*d.timetuple()[:7]))

df['year'] = df['vote-date'].apply(lambda x: x.split('/')[2])
df['month'] = df['vote-date'].apply(lambda x: x.split('/')[1])
df['day'] = df['vote-date'].apply(lambda x: x.split('/')[0])
    
def get_vote_value(v):
    if v == 'Yes':
        return 1
    elif v == 'No':
        return -1
    else:
        return 0

df['vote-value'] = df['vote'].apply(get_vote_value)

In [3]:
df_matrix = pd.DataFrame(index=df['name-en'].unique())
for gn, g in df.groupby('topic_id'):
    df_matrix[gn] = g.set_index('name-en')['vote-value']

In [4]:
#df

In [5]:
df_matrix.shape

(71, 3362)

In [6]:
X = np.matrix(df_matrix.fillna(0).as_matrix()).astype('float')

In [7]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
# X is of dimensions: n_samples x n_features
X_reduced = pca.fit_transform(X)
df_pc = pd.DataFrame(X_reduced, index=df_matrix.index, columns=['PC1', 'PC2', 'PC3'])

In [15]:
#df_pc.sort_values(by='PC1')

In [9]:
X = np.matrix(df_matrix.fillna(0).as_matrix()).astype('float')
X = X - np.mean(X, 0)
pca = PCA(n_components=3)
# X is of dimensions: n_samples x n_features
X_reduced = pca.fit_transform(X)
df_pc = pd.DataFrame(X_reduced, index=df_matrix.index, columns=['PC1', 'PC2', 'PC3'])

In [14]:
#df_pc.sort_values(by='PC1')

In [11]:
pca.components_

array([[-0.02275147, -0.02506966, -0.02567528, ...,  0.01479418,
        -0.02914271,  0.02963961],
       [-0.0824642 , -0.06702808, -0.06871222, ...,  0.05998506,
        -0.02932079,  0.0301413 ],
       [ 0.0301277 ,  0.04760171,  0.04866276, ...,  0.04559627,
         0.00555173, -0.00609546]])

In [12]:
%matplotlib

x = np.array(X_reduced.T[0, :]).astype('float')
y = np.array(X_reduced.T[1, :]).astype('float')
z = np.array(X_reduced.T[2, :]).astype('float')

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, picker=True, s=100)

def onpick(X_3D, event):
    #print(event)
    if hasattr(onpick, '_label'):
        #pass
        onpick._label.remove()
    thisline = event.artist
    ind = event.ind
    #print(type(ind))
    #print(X_3D[0, ind])
    names = df_matrix.iloc[ind].index.values
    #print(names)
    label = ('\n'.join(names))
    pos = (X_3D[0, ind[0]], X_3D[1, ind[0]], X_3D[2, ind[0]])
    #onpick._label = ax.set_title(label
    onpick._label = ax.text(*pos, s=label)
    fig.canvas.draw()
    
fig.canvas.mpl_connect('pick_event', lambda e: onpick(X_reduced.T, e))

Using matplotlib backend: MacOSX


9

In [13]:
matplotlib.rcsetup.interactive_bk

['GTK',
 'GTKAgg',
 'GTKCairo',
 'MacOSX',
 'Qt4Agg',
 'Qt5Agg',
 'TkAgg',
 'WX',
 'WXAgg',
 'CocoaAgg',
 'GTK3Cairo',
 'GTK3Agg',
 'WebAgg',
 'nbAgg']